# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto


## step 1
Installing dependencies 

In [2]:
!pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd

Scraping required data from webpage

In [3]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table', class_='wikitable')
table_rows = table.find_all('tr')

converting scraped data into padas dataframe

In [66]:
# this array will hold the table data
temp = []

# adding invidual subarrays for each table array
for tr in table_rows:
    td = tr.find_all('td')
    row = [d.text.strip() for d in td]

    # excluding rows whose borough is not assigned
    if row and row[1] != 'Not assigned':
        # in case post code don't have neighbourhood assigned
        if row[2] == 'Not assigned':
            row[2] = row[1]
        temp.append(row)

converting array with stored data into dataframe

In [104]:
# creating dataframe out of mentioned array
df = pd.DataFrame(data=temp, columns=['Postcode', 'Borough', 'Neighbourhood'])
print(df.shape)
# creating temporary data frame of neighbourhood seprated by comma's for each postcode
tempNeighbourhoodDf =df.groupby('Postcode')['Neighbourhood'].apply(','.join).reset_index().set_index('Postcode')

#droping rows and joining above dataframes 
df = df.drop('Neighbourhood', axis=1).drop_duplicates().set_index('Postcode')
df = df.join(tempNeighbourhoodDf)
print(df)

(211, 3)
                   Borough                                      Neighbourhood
Postcode                                                                     
M3A             North York                                          Parkwoods
M4A             North York                                   Victoria Village
M5A       Downtown Toronto                           Harbourfront,Regent Park
M6A             North York                    Lawrence Heights,Lawrence Manor
M7A           Queen's Park                                       Queen's Park
...                    ...                                                ...
M8X              Etobicoke        The Kingsway,Montgomery Road,Old Mill North
M4Y       Downtown Toronto                               Church and Wellesley
M7Y           East Toronto  Business Reply Mail Processing Centre 969 Eastern
M8Y              Etobicoke  Humber Bay,King's Mill Park,Kingsway Park Sout...
M8Z              Etobicoke  Kingsway Park South West,Mi

### shape of final dataframe    

In [107]:
# i have made Postcode as my index, that's why i get two rows count in shape of column
print("Shape of final data frame", df.shape)

Shape of final data frame (103, 2)
